# Baysian Regression model- Price Elasticity
- Lecture
- Online source
- Lin paper
- Data Camp
- Basic Baysian inference: Two groups continous outcomes
    - reduced  dataset, treated_b split, only 2013

In [3]:
# Import necessary package 
import pandas as pd
import numpy as np
import seaborn as sn
import tensorflow as tf
import pymc as pm
from scipy import optimize
import statsmodels.api as sm
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
import arviz as az
import tensorflow as tf


# for reproducable results
import random

/opt/conda/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.0' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/tmp/ipykernel_35398/479662750.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
SEED = 5
np.random.seed(SEED)

In [5]:
# 0. Reading-in Set that has missings
water_baysian = pd.read_csv('water_baysian.csv')
water_baysian.dtypes

Unnamed: 0                           int64
year                                 int64
id_c                                 int64
county                              object
state                                int64
grund                              float64
rate_gw                            float64
treated_b                            int64
high_gw_rate                         int64
rate_change                          int64
first_rate_change_year             float64
ever_treated                         int64
rate_change_treatment_indicator      int64
event_time_rate_change             float64
eigengewinnung                     float64
fremdbezug                         float64
n_betriebe_eg                      float64
match_rpf_sa                          bool
match_rpf_sa_eng                      bool
high_gw_cont                       float64
gdp                                float64
gdp_pw                             float64
gdp_pc                             float64
perc_gruene

# 1. Transforming

In [6]:
water_baysian['log_gdp'] = np.log(water_baysian['gdp'])
water_baysian['log_mean_precip'] = np.log(water_baysian['mean_precip'])
water_baysian['log_pop_density'] = np.log(water_baysian['pop_density'])
water_baysian['log_fremdbezug_perfirm'] = np.log(water_baysian['fremdbezug']/water_baysian['n_betriebe_eg'] )
water_baysian['log_perc_gruene'] = np.log(water_baysian['perc_gruene'])
water_baysian['log_rate_gw'] = np.log(water_baysian['rate_gw']+0.0001)
water_baysian['log_sw_area'] = np.log(water_baysian['sw_area'])
# convert to float 32 and round to 3 for staorage saving
water_baysian['log_gdp'] =water_baysian['log_gdp'].round(3).astype('float32')
water_baysian['log_mean_precip'] =water_baysian['log_mean_precip'].round(3).astype('float32')
water_baysian['log_pop_density'] =water_baysian['log_pop_density'].round(3).astype('float32')
water_baysian['log_fremdbezug_perfirm'] = water_baysian['log_fremdbezug_perfirm'].round(3).astype('float32')
water_baysian['log_perc_gruene'] = water_baysian['log_perc_gruene'].round(3).astype('float32')
water_baysian['log_sw_area'] = np.log(water_baysian['sw_area'])
water_baysian['log_sw_area'] = water_baysian['log_sw_area'].round(3).astype('float32')
water_baysian['log_rate_gw'] = water_baysian['log_rate_gw'].round(3).astype('float32')


# 2. Standardizing Data
We are dealing with the nas in the Dataset by masking them at this point and and standardize them according to the process implemented in the Lecture. The masked arrays are just used when we turn to the Baysian Regression.

In [7]:
def standardize_ma(x):
    x_ma = np.ma.masked_invalid(x)
    return (x_ma-x_ma.mean())/x_ma.std()

In [8]:
# Standardize the y variable
log_grund_perfirm = standardize_ma((water_baysian['log_grund_perfirm']))
# Standardize MA the explanatory variables

log_gdp = standardize_ma(water_baysian['log_gdp'])
log_mean_precip = standardize_ma(water_baysian['log_mean_precip'])
log_pop_density = standardize_ma(water_baysian['log_pop_density'])
log_fremdbezug_perfirm = standardize_ma(water_baysian['log_fremdbezug_perfirm'])
log_perc_gruene = standardize_ma(water_baysian['log_perc_gruene'])
log_rate_gw = standardize_ma(water_baysian['log_rate_gw'])
log_sw_area = standardize_ma(water_baysian['log_sw_area'])

In [9]:
# convert to arrays
log_grund_perfirm = np.array(log_grund_perfirm)
log_gdp= np.array(log_gdp)
log_mean_precip= np.array(log_mean_precip)
log_pop_density = np.array(log_pop_density)
log_fremdbezug_perfirm = np.array(log_fremdbezug_perfirm)
log_perc_gruene = np.array(log_perc_gruene)
log_rate_gw = np.array(water_baysian['log_rate_gw'])
log_sw_area = np.array(log_sw_area)

# 3. Baysian Regression 
 - SOO
 - RPF & SA
 - uninformed prior: 

In [10]:
with pm.Model() as bay_model1:
    # Priors
    constant = pm.Normal('contant', mu =0.0 , sigma = 1.0)
    σ_prior = 0.1
    # explanatory
    b_log_gdp = pm.Normal('b_log_gdp', mu = 0.0,sigma =σ_prior )
    b_log_mean_precip = pm.Normal('b_log_mean_precip', mu = 0.0,sigma =σ_prior )
    b_log_pop_density = pm.Normal('b_log_pop_density', mu = 0.0,sigma =σ_prior )
    b_log_fremdbezug_perfirm = pm.Normal('b_log_fremdbezug_perfirm', mu = 0.0,sigma =σ_prior )
    b_log_perc_gruene = pm.Normal('b_log_perc_gruene', mu = 0.0,sigma =σ_prior )
    b_log_rate_gw = pm.Normal('b_log_rate_gw', mu = 0.0,sigma =σ_prior )
    b_log_sw_area = pm.Normal('b_log_sw_area', mu=0.0, sigma=σ_prior)
    
    # draw of missings
    Log_gdp = pm.Normal('Log_gdp',mu =0,sigma =1.0 , observed = log_gdp)
    Log_pop_density = pm.Normal('Log_pop_density',mu =0,sigma =1.0 , observed = log_pop_density )
    Log_mean_precip = pm.Normal('Log_mean_precip',mu =0,sigma =1.0 , observed = log_mean_precip)
    Log_mean_fremdbezug_perfirm  = pm.Normal('Log_mean_fremdbezug_perfirm',mu =0,sigma =1.0 , observed = log_fremdbezug_perfirm)
    Log_mean_perc_gruene  = pm.Normal('Log_mean_perc_gruene',mu =0,sigma =1.0 , observed = log_perc_gruene)
    Log_rate_gw = pm.Normal('Log_rate_gw', mu =0,sigma =1.0, observed=log_rate_gw )
    Log_sw_area = pm.Normal('Log_sw_area', mu=0, sigma=1.0, observed=log_sw_area)
    
    # Model
    μ = constant + b_log_rate_gw*Log_rate_gw + b_log_gdp*Log_gdp+ b_log_sw_area * Log_sw_area + b_log_mean_precip*Log_mean_precip + b_log_pop_density*Log_pop_density + b_log_fremdbezug_perfirm * Log_mean_fremdbezug_perfirm+b_log_perc_gruene*Log_mean_perc_gruene
    σ = pm.HalfNormal('σ', 1)
    log_y = pm.Normal('log_y', μ,σ, observed =log_grund_perfirm)
    # for now reduced comp

/opt/conda/lib/python3.9/site-packages/pymc/model/core.py:1323: ImputationWarning: Data in Log_gdp contains missing values and will be automatically imputed from the sampling distribution.
  warnings.warn(impute_message, ImputationWarning)
/opt/conda/lib/python3.9/site-packages/pymc/model/core.py:1323: ImputationWarning: Data in Log_pop_density contains missing values and will be automatically imputed from the sampling distribution.
  warnings.warn(impute_message, ImputationWarning)
/opt/conda/lib/python3.9/site-packages/pymc/model/core.py:1323: ImputationWarning: Data in Log_mean_precip contains missing values and will be automatically imputed from the sampling distribution.
  warnings.warn(impute_message, ImputationWarning)
/opt/conda/lib/python3.9/site-packages/pymc/model/core.py:1323: ImputationWarning: Data in Log_mean_fremdbezug_perfirm contains missing values and will be automatically imputed from the sampling distribution.
  warnings.warn(impute_message, ImputationWarning)
/opt

In [11]:
with bay_model1:
    trace_bay_model1 = pm.sample(draws=500, cores=1)
    pm.save_trace(trace_bay_model1, directory="my_trace_folder", overwrite=True)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (2 chains in 1 job)
NUTS: [contant, b_log_gdp, b_log_mean_precip, b_log_pop_density, b_log_fremdbezug_perfirm, b_log_perc_gruene, b_log_rate_gw, b_log_sw_area, Log_gdp_unobserved, Log_pop_density_unobserved, Log_mean_precip_unobserved, Log_mean_fremdbezug_perfirm_unobserved, Log_sw_area_unobserved, σ, log_y_unobserved]


Sampling 2 chains for 1_000 tune and 500 draw iterations (2_000 + 1_000 draws total) took 214 seconds.
We recommend running at least 4 chains for robust computation of convergence diagnostics


AttributeError: module 'pymc' has no attribute 'save_trace'

In [12]:
trace_bay_model1.to_netcdf('baysian_elast_trace.nc')

'baysian_elast_trace.nc'

In [ ]:
idata = az.from_netcdf("baysian_elast_trace.nc")

In [13]:
variables_ofinterest = ['b_log_rate_gw','b_log_gdp', 'b_log_mean_precip', 'b_log_pop_density','b_log_fremdbezug_perfirm', 'b_log_perc_gruene' ,'b_log_sw_area']
summary_pm_elast = pm.summary(trace_bay_model1, var_names=variables_ofinterest,  round_to=3)
summary_pm_elast

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
b_log_rate_gw,-0.009,0.007,-0.021,0.005,0.000,0.000,1166.574,871.841,1.000
b_log_gdp,0.082,0.033,0.019,0.145,0.001,0.001,1214.744,943.961,1.002
b_log_mean_precip,0.024,0.023,-0.019,0.067,0.001,0.000,1851.270,907.527,1.001
b_log_pop_density,0.083,0.038,0.015,0.159,0.001,0.001,1142.072,846.396,1.000
b_log_fremdbezug_perfirm,0.303,0.024,0.257,0.344,0.001,0.000,1397.570,955.805,1.001
b_log_perc_gruene,0.002,0.025,-0.044,0.047,0.001,0.001,1541.347,850.496,0.999
b_log_sw_area,0.215,0.033,0.149,0.275,0.001,0.001,1083.383,794.830,0.998


In [ ]:
az.plot_trace(trace_bay_model1);

 - Check Assumptions and Model FIt

In [14]:

summary_pm_elast.reset_index(inplace = True)

summary_pm_elast.rename(columns= {'index':'Variable','mean':'Mean', 'sd':'SD'}, inplace = True)
summary_pm_elast = summary_pm_elast.round(4)
summary_pm_elast['r_hat'] = summary_pm_elast['r_hat'].round(2)
col_to_drop = summary_pm_elast.columns[5:9]
summary_pm_elast= summary_pm_elast.drop(columns=col_to_drop, axis = 1)

In [15]:
latex_df = summary_pm_elast.to_latex(index = False, caption = 'Summary Baysian - Elasticity', float_format= "%.3f", escape=True)
with open('summary_rate_change_bay', 'w') as file:
    file.write(latex_df)